다음은 미국 동부지역의 환자 7만명의 항생제 투여와 클로스트리움 디피실균의 감염 여부를 기록한 데이터다. 
항생제 투여 정보는 일 단위로 기록되어 있고, 하루에 2회 이상 투여하는 경우도 존재한다.
- 데이터: Antibiotic_70K_patinet.txt(구분자: ","(comma), 70,000 row, 8 columns)
- ID: 환자 고유 번호(숫자형)
- total_d: 내원기간 중 항생제 누적 투여일수(숫자형)
- hight_d: 내원기간 중 고위험군 항생제 누적 투여일수 (숫자형)
- low_d: 내원기간 중 저위험군 항생제 누적 투여일수 (숫자형)
- non_high_d: 고위험군 항생제 투여일수를 제외한 항생제 누적 투여일 수 (숫자형)
- high_p: 고위험군 항생제 투여 비중 (숫자형)
- cdiff:클로스트리디움 디피실균의 감염 여부 (숫자형)
- age_5: 나이대, 20세부터 5세 단위로 (숫자형)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

## Data read and preprocessing

In [3]:
data = pd.read_csv('Antibiotic_70K_patinets.txt')

In [7]:
print(type(data), data.shape, type(data.high_p[0]))

<class 'pandas.core.frame.DataFrame'> (70000, 8) <class 'numpy.float64'>


In [5]:
data.head()

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
0,1,2,1,2,1,50.00,1,1
1,2,25,9,16,16,36.00,1,5
2,3,6,0,6,6,0.00,1,4
3,4,7,6,6,1,85.71,1,4
4,5,4,3,2,1,75.00,1,3


In [19]:
# Descriptive statics
data.describe()

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.00000
mean,35000.500000,8.049771,3.774386,6.131871,4.275386,42.254031,0.021471,2.74870
std,20207.403759,13.885638,8.795789,11.502711,7.950063,39.836986,0.144951,1.56102
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000
25%,17500.750000,2.000000,0.000000,2.000000,1.000000,0.000000,0.000000,1.00000
50%,35000.500000,4.000000,2.000000,3.000000,2.000000,37.500000,0.000000,3.00000
75%,52500.250000,8.000000,4.000000,6.000000,4.000000,80.000000,0.000000,4.00000
max,70000.000000,526.000000,368.000000,397.000000,294.000000,100.000000,1.000000,7.00000


### 문제1) 각 나이대별로 저 위험군 항생제 투여일수를 확인하고자 한다. 저위험군 항생제 투여 일수 비중 평균이 가장 낮은 나이대(age_5)를 기술하시오

In [20]:
grouped = data['low_d'].groupby(data['age_5'])

In [23]:
grouped.mean()
# grouped.size()
# grouped.sum()

age_5
1      1.532426
2      2.213510
3      3.239355
4      5.685472
5     15.670451
6     52.831301
7    122.588745
Name: low_d, dtype: float64

### 문제2) 분석에 앞서 결측치와 이상치를 확인하고자 한다.
- 각 변수별 결측치 개수와 총합을 구하시오

[정답] 
- null count 0
- 1258

In [24]:
# Check the missing value
np.sum(pd.isnull(data))

ID             0
total_d        0
high_d         0
low_d          0
none_high_d    0
high_p         0
cdiff          0
age_5          0
dtype: int64

- 항생제 누적투여일수(total_d) 값이 이상치를 보이는 환자의 수를 구하시오
- 이상치의 기준은 아래와 같이 정의하고 평균과 표준편차는 소수점 둘째 자리에서 버림하여 첫째 자리까지 사용
- 이상치: X>평균 + 3*표준편차 또는 X<평균-3*표준편차

In [26]:
def count_outlier(x):
    mean_x = np.mean(x)    # 평균
    sd_x = np.std(x)       # 표준편차
    over_three_sigma = mean_x + 3*sd_x
    under_three_sigma = mean_x - 3*sd_x
    count_under_3sigma = len(x[x<under_three_sigma])    # 3표준편차 보다 작은 갯수
    count_over_3sigma = len(x[x>over_three_sigma])      # 3표준편차 보다 큰 갯수
    count_outlier_obs = count_under_3sigma + count_over_3sigma    # 아웃라이어 갯수
    
    return(count_outlier_obs)

In [28]:
# 모든 데이터의 열 계산
# data.apply -> 각 열 또는 행 별로 함수를 진행하라
result = data.apply(lambda x:count_outlier(x)).to_frame(name='result').reset_index()

In [29]:
result

,index,result
0,ID,0
1,total_d,1258
2,high_d,1095
3,low_d,1211
4,none_high_d,1157
5,high_p,0
6,cdiff,1503
7,age_5,0


### 문제3) 환자의 나이대(age_5)와 고위험군 항생제 투여 비중의 상관관계를 분석하고자 한다. Pearson와 Spearman의 상관관계를 소수점 셋째 자리 이하는 버리고 소수점 둘째 짜리까지 각각 서술하시오.

[정답]
- Pearson: 0.13
- Spearman: 0.17

In [38]:
data[['high_p', 'age_5']].corr(method='pearson')

,high_p,age_5
high_p,1.000000,0.125046
age_5,0.125046,1.000000


In [39]:
data[['high_p', 'age_5']].corr(method='spearman')

,high_p,age_5
high_p,1.000000,0.167689
age_5,0.167689,1.000000


### 문제 4)클로스트리디움 디피실균의 감염여부에 따른 고위험군 항생제 투여 비중 차이를 알아보고자 한다. 감염에 따라 집단을 나누고 고위험군 항생제 투여 비중 평균값의 차이를 적절한 통계방법론을 사용하여 1% 유의수준에서 검정한 다음 아래의 질문에 답 하시오
- 귀무가설 기각여부를 Y N으로 기술하고, 분산 동질성 여부를 Y N으로 기술하시오
- 검정 통계량의 절대값을 소수점 첫째자리 까지 기술하시오(소수점 둘째 자리 미만은 버림)

비고) 독립표본 t test 의 두 정규분포의 분산값이 같은경우와 같지 않은 경우에 사용하는 검정 통계량이 다르기 때문에 
equal_var 을 사용하여 지정해줘야한다. (default 값은 True)

[정답]
- 귀무가설 기각여부: Y
- 분산 동질성 여부: N
- 검정 통계량: 5.1

In [61]:
data[data['cdiff'] == 1].high_p.mean()

46.94235528942116

In [62]:
data[data['cdiff'] == 1].high_p.mean()

42.15115742295283

In [75]:
t, p = stats.ttest_ind(data[data['cdiff'] == 1].high_p, data[data['cdiff'] == 0].high_p, equal_var=False)

In [76]:
t, p
print('t-test is ', format(t, '.1f'), 'p-value is ', p)

t-test is  5.1 p-value is  3.623790784540754e-07


In [77]:
t, p = stats.ttest_ind(data[data['cdiff'] == 1].high_p, data[data['cdiff'] == 0].high_p, equal_var=True)

In [78]:
t, p
print('t-test is ', format(t, '.1f'), 'p-value is ', p)

t-test is  4.6 p-value is  3.975323449530875e-06


### 5)클로스트리디움 디피실균의 감염여부를 종속변수로 하고, 환자 식별과(ID)와 "고위험군 항생제 투여일수를 제외한 항생제 누적투여(none_high_d)를제외한 모든 변수(총 5개)를 독립변수로 하여 로지스틱 회귀분석을 실시하라.
- Logistic regression 설정 값이
- Inverse of Regularization = 100000, seed = 1234, Penalty = l2, solver = newton-cg
- 주의사항: 설정 값을 달리할 경우 다른 결과가 나올 수 있음
- 다른 모든 결과가 동일할 때 감염 여부의 오즈비(p/1-p)에 가장 큰 영향을 미치는 변수를 서술하시오

- C : Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
- random_state : random 데이터 생성 시 seed of random number

[정답] age_5

In [94]:
model = LogisticRegression(C=100000, random_state=1234, fit_intercept=True, penalty='l2', solver='newton-cg')

In [100]:
x = data[['total_d', 'high_d', 'low_d', 'high_p', 'age_5']]
y = data['cdiff']

In [104]:
x2 = sm.add_constant(x)

In [105]:
model = sm.OLS(y, x2)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  cdiff   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     242.7
Date:                Sat, 22 Aug 2020   Prob (F-statistic):          5.79e-258
Time:                        13:58:33   Log-Likelihood:                 36472.
No. Observations:               70000   AIC:                        -7.293e+04
Df Residuals:                   69994   BIC:                        -7.288e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0049      0.001     -3.873      0.0